<a href="https://colab.research.google.com/github/Jamunajayanna/DataScience/blob/main/EL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset
!kaggle datasets download -d fedesoriano/stroke-prediction-dataset

# Unzip the dataset
!unzip stroke-prediction-dataset.zip


cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset
License(s): copyright-authors
  0% 0.00/67.4k [00:00<?, ?B/s]
100% 67.4k/67.4k [00:00<00:00, 74.6MB/s]
Archive:  stroke-prediction-dataset.zip
  inflating: healthcare-dataset-stroke-data.csv  


In [2]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [3]:
# Load the dataset
df = pd.read_csv('/content/healthcare-dataset-stroke-data.csv')

# Show the first few rows
df.head()


,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [4]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE

warnings.filterwarnings("ignore")

# Load dataset
df = pd.read_csv("healthcare-dataset-stroke-data.csv")

# Drop unnecessary column
df.drop(columns=['id'], inplace=True)

# Encode categorical variables
categorical_columns = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']
label_encoders = {}

for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Fill missing values with median
df.fillna(df.median(), inplace=True)

# Define features and target
X = df.drop(columns=['stroke'])
y = df['stroke']

# Print class distribution
print("Class distribution before SMOTE:\n", y.value_counts())

# Balance the dataset using SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Print class distribution after SMOTE
print("Class distribution after SMOTE:\n", y_resampled.value_counts())

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Train RandomForestClassifier instead of Decision Tree
clf = RandomForestClassifier(random_state=42, class_weight="balanced", n_estimators=200)
clf.fit(X_train, y_train)

# Evaluate model
y_pred = clf.predict(X_test)
print("\nModel Evaluation:\n", classification_report(y_test, y_pred))

# Function to get user input and predict stroke
def predict_stroke():
    try:
        age = int(input("Enter Age: "))
        hypertension = int(input("Hypertension (0: No, 1: Yes): "))
        heart_disease = int(input("Heart Disease (0: No, 1: Yes): "))
        diabetes = int(input("Diabetes (0: No, 1: Yes): "))
        smoking = int(input("Smoking Status (0: Never, 1: Former, 2: Smokes): "))

        if smoking not in [0, 1, 2]:
            print("Invalid smoking status. Use 0, 1, or 2.")
            return

        # Assign default values for missing categorical features
        default_gender = label_encoders['gender'].transform(["Male"])[0]
        default_ever_married = label_encoders['ever_married'].transform(["Yes"])[0]
        default_work_type = label_encoders['work_type'].transform(["Private"])[0]
        default_residence_type = label_encoders['Residence_type'].transform(["Urban"])[0]

        # Use average values for numerical features
        avg_glucose_level = X_train['avg_glucose_level'].mean()
        bmi = X_train['bmi'].mean()

        # Create DataFrame for prediction
        input_df = pd.DataFrame([[
            default_gender, age, hypertension, heart_disease,
            default_ever_married, default_work_type, default_residence_type,
            avg_glucose_level, bmi, smoking
        ]], columns=X.columns)

        # Predict
        predicted_stroke = clf.predict(input_df)

        # Show result
        if predicted_stroke[0] == 1:
            result = "The patient is likely to have a stroke."
        else:
           result = "The patient is unlikely to have a stroke."

        print("\nPrediction:", result)




    except ValueError:
        print("Invalid input. Please enter numeric values only.")

# Run prediction function
predict_stroke()


Class distribution before SMOTE:
 stroke
0    4861
1     249
Name: count, dtype: int64
Class distribution after SMOTE:
 stroke
1    4861
0    4861
Name: count, dtype: int64

Model Evaluation:
               precision    recall  f1-score   support

           0       0.96      0.93      0.95       975
           1       0.93      0.96      0.95       970

    accuracy                           0.95      1945
   macro avg       0.95      0.95      0.95      1945
weighted avg       0.95      0.95      0.95      1945

Enter Age: 24
Hypertension (0: No, 1: Yes): 1
Heart Disease (0: No, 1: Yes): 1
Diabetes (0: No, 1: Yes): 1
Smoking Status (0: Never, 1: Former, 2: Smokes): 2

Prediction: The patient is unlikely to have a stroke.


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight

# Load dataset
df = pd.read_csv('healthcare-dataset-stroke-data.csv')

# Drop 'id' column
df.drop(columns=['id'], inplace=True)

# Encode categorical columns
categorical_columns = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']
for col in categorical_columns:
    df[col] = LabelEncoder().fit_transform(df[col])

# Handle missing values
df.fillna(df.median(), inplace=True)

# Define features (X) and target (y)
X = df.drop(columns=['stroke'])
y = df['stroke']

# Check class imbalance
stroke_counts = y.value_counts()
print("Class distribution:\n", stroke_counts)

# Compute class weights to handle imbalance
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Split dataset into training & testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Train Random Forest Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight=class_weight_dict)
clf.fit(X_train, y_train)

# Function to predict stroke
def predict_stroke():
    try:
        # Get user input
        age = int(input("Enter Age: "))
        hypertension = int(input("Hypertension (0: No, 1: Yes): "))
        heart_disease = int(input("Heart Disease (0: No, 1: Yes): "))
        diabetes = int(input("Diabetes (0: No, 1: Yes): "))
        smoking = int(input("Smoking Status (0: Never, 1: Former, 2: Smokes): "))

        # Validate smoking input
        if smoking not in [0, 1, 2]:
            print("Invalid smoking status. Use 0, 1, or 2.")
            return

        # Prepare input for prediction
        input_array = np.array([[age, hypertension, heart_disease, diabetes, smoking]])

        # Adjust input shape if needed
        missing_features = X_train.shape[1] - input_array.shape[1]
        if missing_features > 0:
            input_array = np.append(input_array, np.zeros((1, missing_features)), axis=1)

        # Make prediction
        predicted_stroke = clf.predict(input_array)

        # Show result
        if predicted_stroke[0] == 1:
            result = "The patient is likely to have a stroke."
        else:
            result = "The patient is unlikely to have a stroke."

        print("\nPrediction:", result)

    except ValueError:
        print("Invalid input. Please enter numeric values only.")

# Run prediction
predict_stroke()


Class distribution:
 stroke
0    4861
1     249
Name: count, dtype: int64
Enter Age: 75
Hypertension (0: No, 1: Yes): 1
Heart Disease (0: No, 1: Yes): 1
Diabetes (0: No, 1: Yes): 1
Smoking Status (0: Never, 1: Former, 2: Smokes): 2

Prediction: The patient is unlikely to have a stroke.
